# RAG Pipeline - Retrieval-Augmented Generation

Пайплайны для:
- Vector database (FAISS, Chroma)
- Retrieval + LLM
- Document QA
- Semantic search

In [ ]:
!pip install langchain chromadb faiss-cpu sentence-transformers transformers torch -q

In [ ]:
import pandas as pd
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS, Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, CSVLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

## 1. Загрузка документов

In [ ]:
# === ВАШИ ДОКУМЕНТЫ ===
# Вариант 1: Текстовые файлы
# loader = TextLoader('document.txt')
# documents = loader.load()

# Вариант 2: CSV с текстами
# loader = CSVLoader('documents.csv', csv_args={'delimiter': ','})
# documents = loader.load()

# Вариант 3: Список текстов
from langchain.schema import Document

texts = [
    "Python - это высокоуровневый язык программирования.",
    "Machine Learning - это подмножество искусственного интеллекта.",
    "Deep Learning использует нейронные сети с множеством слоев.",
    # Добавьте свои документы
]

documents = [Document(page_content=text) for text in texts]
print(f"Загружено документов: {len(documents)}")

## 2. Разделение на chunks

In [ ]:
# Разбиение длинных документов на части
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
)

chunks = text_splitter.split_documents(documents)
print(f"Создано chunks: {len(chunks)}")

## 3. Создание embeddings и vector store

In [ ]:
# Модель для embeddings
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
# Альтернативы: "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2" (для русского)

embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
    model_kwargs={'device': device}
)

print("✓ Embedding модель загружена!")

In [ ]:
# Создание FAISS векторной базы
vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)

# Сохранение
vectorstore.save_local("faiss_index")

# Загрузка существующей базы
# vectorstore = FAISS.load_local("faiss_index", embeddings)

print("✓ Vector store создан!")

## 4. Semantic Search (без LLM)

In [ ]:
# Поиск релевантных документов
query = "Что такое Python?"

# Топ-3 релевантных документа
relevant_docs = vectorstore.similarity_search(query, k=3)

print(f"Запрос: {query}\n")
for i, doc in enumerate(relevant_docs, 1):
    print(f"{i}. {doc.page_content}\n")

In [ ]:
# Поиск с оценкой similarity score
docs_with_scores = vectorstore.similarity_search_with_score(query, k=3)

print(f"Запрос: {query}\n")
for doc, score in docs_with_scores:
    print(f"Score: {score:.4f}")
    print(f"Content: {doc.page_content}\n")

## 5. RAG с LLM

In [ ]:
# Загрузка LLM модели
LLM_MODEL = "microsoft/phi-2"
# Альтернативы: "google/flan-t5-base", "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# Pipeline для генерации
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7,
)

llm = HuggingFacePipeline(pipeline=text_generator)
print("✓ LLM загружена!")

In [ ]:
# Создание RAG chain
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

print("✓ RAG chain создан!")

In [ ]:
# Задать вопрос
query = "Что такое Machine Learning?"

result = qa_chain({"query": query})

print(f"Вопрос: {query}\n")
print(f"Ответ: {result['result']}\n")
print("Источники:")
for i, doc in enumerate(result['source_documents'], 1):
    print(f"{i}. {doc.page_content}")

## 6. Batch предсказания для соревнования

In [ ]:
# === ТЕСТОВЫЕ ВОПРОСЫ ===
test_df = pd.read_csv('test_questions.csv')  # Колонка: question

answers = []
for question in test_df['question']:
    result = qa_chain({"query": question})
    answers.append(result['result'])

# Submission
submission = pd.DataFrame({
    'id': test_df.index if 'id' not in test_df.columns else test_df['id'],
    'answer': answers
})

submission.to_csv('rag_submission.csv', index=False)
print("✓ Submission сохранен!")

## 7. Использование Chroma вместо FAISS

In [ ]:
# Альтернатива: ChromaDB
# chroma_db = Chroma.from_documents(
#     documents=chunks,
#     embedding=embeddings,
#     persist_directory="./chroma_db"
# )

# # Загрузка существующей базы
# chroma_db = Chroma(
#     persist_directory="./chroma_db",
#     embedding_function=embeddings
# )

# retriever = chroma_db.as_retriever()